In [5]:
import os
import numpy as np
import pandas as pd
from glob import glob
from termcolor import colored
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras import layers, models, optimizers, losses

from sklearn.model_selection import train_test_split as tts

In [28]:
batch_size = 32
EPOCHS = 50
size = 160
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001

In [23]:
root_data = glob("D:/Dataset/dataset/KAMP/01. Dataset_OCR/*")

df = pd.DataFrame();
img_path = []; img_label = []
for i in tqdm(root_data):
    label_temp = i[-1]
    for j in glob(i+"/*"):
        img_path.append(j)
        img_label.append(label_temp)
df["img_path"]=img_path
df["img_label"]=img_label
df["img_label"] = df["img_label"].astype(int)

train_df, test_df = tts(df, test_size=0.2, stratify=df["img_label"])
valid_df, test_df = tts(test_df, test_size=0.5, stratify=test_df["img_label"])

train_ds = tf.data.Dataset.from_tensor_slices((train_df["img_path"].values, train_df["img_label"].values))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_df["img_path"].values, valid_df["img_label"].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df["img_path"].values, test_df["img_label"].values))

  0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def decode_img(img):
    img = tf.io.decode_jpeg(img, channels=3)
    return tf.image.resize(img, [size, size])

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=100)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds
def configure_for_performance_test(ds):
    ds = ds.cache()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

In [13]:
def build_eff_model():
    # Top
    x = layers.Input(shape=(size, size, 3))
    
    # aug
    y = layers.RandomRotation(factor=0.02)(x)
    y = layers.RandomZoom(height_factor=0.1, width_factor=0.1)(y)
    
    # base
    y = tf.keras.applications.EfficientNetB0(input_shape=(size,size,3), include_top=False, weights='imagenet')(y)

    # Rebuild
    dropout_rate = 0.4
    y = layers.GlobalAveragePooling2D(name="avg_pool")(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(dropout_rate)(y)
    y = layers.Dense(128, activation="leaky_relu")(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(dropout_rate)(y)
    y = layers.Dense(10, activation="softmax", name="output")(y)
    
    # compile
    model = models.Model(x, y, name="EfficientNet")
    return model
model = build_eff_model()

In [15]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tfa.optimizers.AdamW(learning_rate=LEARNING_RATE,weight_decay=WEIGHT_DECAY)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [16]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [17]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [24]:
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
valid_ds = valid_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = configure_for_performance(train_ds)
valid_ds = configure_for_performance_test(valid_ds)

In [31]:
for epoch in tqdm(range(EPOCHS)):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for idx, (images, labels) in enumerate(tqdm(train_ds)):
        train_step(images, labels)
        
        if idx % 100 == 0:
            print(f'step {idx + 1}, '
                  f'Loss: {train_loss.result()}, '
                  f'Accuracy: {train_accuracy.result() * 100}, ')
        
        
    for test_images, test_labels in valid_ds:
        test_step(test_images, test_labels)

        
    print(f'epochs {epoch + 1}, '
          f'Loss: {train_loss.result()}, '
          f'Accuracy: {train_accuracy.result() * 100}, '
          f'Test Loss: {test_loss.result()}, '
          f'Test Accuracy: {test_accuracy.result() * 100}')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/681 [00:00<?, ?it/s]

step 1, Loss: 0.003166596405208111, Accuracy: 100.0, 
step 101, Loss: 0.057739388197660446, Accuracy: 98.63861083984375, 
step 201, Loss: 0.05763911083340645, Accuracy: 98.58519744873047, 
step 301, Loss: 0.053594570606946945, Accuracy: 98.65032958984375, 
step 401, Loss: 0.05193837359547615, Accuracy: 98.69857025146484, 
step 501, Loss: 0.05008139833807945, Accuracy: 98.76496887207031, 
step 601, Loss: 0.050856005400419235, Accuracy: 98.7676773071289, 
epochs 1, Loss: 0.05266738682985306, Accuracy: 98.74707794189453, Test Loss: 0.04534342139959335, Test Accuracy: 99.30249786376953


  0%|          | 0/681 [00:00<?, ?it/s]

step 1, Loss: 0.004964425228536129, Accuracy: 100.0, 
step 101, Loss: 0.04876583069562912, Accuracy: 98.79331970214844, 
step 201, Loss: 0.048268988728523254, Accuracy: 98.7251205444336, 
step 301, Loss: 0.05039265751838684, Accuracy: 98.69186401367188, 
step 401, Loss: 0.05121751129627228, Accuracy: 98.6829833984375, 
step 501, Loss: 0.048297442495822906, Accuracy: 98.74002075195312, 
step 601, Loss: 0.049156464636325836, Accuracy: 98.7676773071289, 
epochs 2, Loss: 0.04779919981956482, Accuracy: 98.8067398071289, Test Loss: 0.036789920181035995, Test Accuracy: 99.30249786376953


...

  0%|          | 0/681 [00:00<?, ?it/s]

step 1, Loss: 0.0015251018339768052, Accuracy: 100.0, 
step 101, Loss: 0.046546854078769684, Accuracy: 98.91707611083984, 
step 201, Loss: 0.04515640810132027, Accuracy: 98.84950256347656, 
step 301, Loss: 0.04927374795079231, Accuracy: 98.7437744140625, 
step 401, Loss: 0.047716088593006134, Accuracy: 98.75312042236328, 
step 501, Loss: 0.04554793983697891, Accuracy: 98.77120971679688, 
step 601, Loss: 0.04644201323390007, Accuracy: 98.77808380126953, 
epochs 5, Loss: 0.046040669083595276, Accuracy: 98.82051086425781, Test Loss: 0.04204815253615379, Test Accuracy: 99.1923599243164


  0%|          | 0/681 [00:00<?, ?it/s]

step 1, Loss: 0.011221538297832012, Accuracy: 100.0, 
step 101, Loss: 0.04010654240846634, Accuracy: 99.19554901123047, 


KeyboardInterrupt: 